# 과정
* 1.  특정 합성곱 층의 한 필터 값을 최대화하는 손실 함수를 정의한다.
> 이 활성화 값을 최대화하기 위해 입력 이미지를 변경하도록 확률적 경사 상승법을 사용한다. 예를 들어 여기에선 ImageNet에 사전 훈련된 VGG16 네트워크에서 block3_conv1 층 필터 0번의 활성화를 손실로 정의한다.
> ### 필터 시각화를 위한 손실 텐서 정의하기

In [57]:
from keras.applications import VGG16
from keras import backend as K

model = VGG16(weights='imagenet',
              include_top=False)

layer_name = 'block3_conv1'
filter_index = 0

layer_output = model.get_layer(layer_name).output
loss = K.mean(layer_output[:, :, :, filter_index])

> 경사 상승법을 구현하기 위해 모델의 입력에 대한 손실의 그래디언트가 필요하다. 이를 위해 케라스의 backend 모듈에 있는 gradients 함수를 사용한다.
> ### 입력에 대한 손실의 그래디언트 구하기

In [ ]:
import tensorflow as tf
grads = K.gradients(loss, model.input)[0]

> 경사 상승법 과정을 부드럽게 하기 위해 사용하는 한 가지 기법은 그래디언트 텐서를 L2 노름(텐서에 있는 값을 제곱한 합의 제곱근)로 나누어 정규화하는 것이다.
> ### 그래디언트 정규화하기

In [ ]:
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5) # 0나눗셈을 방지하기 위해 1e - 5를 더한다

> 주어진 입력 이미지에 대해 손실 텐서와 그래디언트 텐서를 계산해야 한다. 
> ```iterate```는 넘파이 텐서(크기가 1인 텐서의 리스트)를 입력으로 받아 손실과 그래디언트 2개의 넘파이 텐서를 반환한다.
> ### 입력 값에 대한 넘파이 출력 값 추출하기

In [ ]:
iterate = K.function([model.input], [loss, grads])

import numpy as np
loss_value, grads_value = iterate([np.zeros((1, 150, 150, 3))])

> ### 확률적 경사 상승법을 사용한 손실 최대화하기

In [ ]:
input_img_data = np.random.random((1, 150, 150, 3)) * 20 + 128. # 잡음이 섞인 회색 이미지로 시작

step = 1. # 업데이트할 그래디언트의 크기
for i in range(40): # 경사 상승법을 40회 실행
    loss_value, grads_value = iterate([input_img_data])
    # 손실과 그래디언트 계산
    input_img_data += grads_value * step
    # 손실을 최대화하는 방향으로 입력이미지를 수정한다.

> 결과 이미지 텐서는 (1, 150, 150, 3)크기의 부동 소수 텐서이기 때문에 후처리할 필요가 있다.
> ### 텐서를 이미지 형태로 변환하기 위한 유틸리티 함수

In [ ]:
def deprocess_image(x):
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1 # 텐서의 평균이 0, 표준편차가 0.1이 되도록 정규화한다.
    
    x += 0.5
    x = np.clip(x, 0, 1) # [0, 1]로 클리핑한다.
    
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x # RGB 배열로 변환

> ### 필터 시각화 이미지를 만드는 함수

In [ ]:
import numpy as np
def generate_pattern(layer_name, filter_index, size=150):
    # 주어진 층과 필터의 활성화를 최대화하기 위한 손실 함수를 정의합니다
    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:, :, :, filter_index])

    # 손실에 대한 입력 이미지의 그래디언트를 계산합니다
    grads = K.gradients(loss, model.input)[0]

    # 그래디언트 정규화
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

    # 입력 이미지에 대한 손실과 그래디언트를 반환합니다
    iterate = K.function([model.input], [loss, grads])

    # 잡음이 섞인 회색 이미지로 시작합니다
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.

    # 경사 상승법을 40 단계 실행합니다
    step = 1.
    for i in range(40):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

    img = input_img_data[0]
    return deprocess_image(img)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(generate_pattern('block3_conv1', 0))

> ### 층에 있는 각 필터에 반응하는 패턴 생성하기

In [ ]:
for layer_name in ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1']:
    size = 64
    margin = 5

    # 결과를 담을 빈 (검은) 이미지
    results = np.zeros((8 * size + 7 * margin, 8 * size + 7 * margin, 3), dtype='uint8')

    for i in range(8):  # results 그리드의 행을 반복합니다
        for j in range(8):  # results 그리드의 열을 반복합니다
            # layer_name에 있는 i + (j * 8)번째 필터에 대한 패턴 생성합니다
            filter_img = generate_pattern(layer_name, i + (j * 8), size=size)

            # results 그리드의 (i, j) 번째 위치에 저장합니다
            horizontal_start = i * size + i * margin
            horizontal_end = horizontal_start + size
            vertical_start = j * size + j * margin
            vertical_end = vertical_start + size
            results[horizontal_start: horizontal_end, vertical_start: vertical_end, :] = filter_img

    # results 그리드를 그립니다
    plt.figure(figsize=(20, 20))
    plt.imshow(results)
    plt.show()

> 이런 필터 시각화를 통해 컨브넷 층이 바라보는 방식을 이해할 수 있다. 컨브넷의 각 층은 필터의 조합으로 입력을 표현할 수 있는 일련의 필터를 학습한다. 이는 푸리에 변환을 사용하여 신호를 일련의 코사인 함수로 분해할 수 있는 것과 비슷하다. 이 컨브넷 필터들은 모델의 상위층으로 갈수록 점점 더 복잡해지고 개선된다.
* 모델에 있는 첫 번째 층의 필터는 간단한 대각선 방향의 에지와 색깔(또는 색깔이 있는 에지)를 인코딩한다.
* block2_conv1의 필터는 에지나 색깔의 조합으로 만들어진 간단한 질감을 인코딩한다.
* 더 상위층의 필터는 깃털 , 눈, 나뭇잎 등 자연적인 이미지에서 찾을 수 있는 질감을 닮아 가기 시작한다.